# Data Engineering

In [18]:
import pandas as pd

In [19]:
FILE_PATH_GAMES = '../data/nbaBettingData.csv'
FILE_PATH_DETAILED_GAMES = '../data/detailedNbaGamesData.csv'
FILE_PATH_RANKINGS = '../data/ranking.csv'

gamesDf = pd.read_csv(FILE_PATH_GAMES)
dGamesDf = pd.read_csv(FILE_PATH_DETAILED_GAMES)
rankingsDf = pd.read_csv(FILE_PATH_RANKINGS)

/var/folders/19/48sc_t714fz_96wrj2kckpy80000gn/T/ipykernel_53586/3547755855.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dGamesDf = pd.read_csv(FILE_PATH_DETAILED_GAMES)


In [20]:
totalPts = gamesDf.PTS_home + gamesDf.PTS_away
gamesDf["PTS_total"] = totalPts

In [21]:
gamesDf["GAME_DATE_EST"] = pd.to_datetime(gamesDf["GAME_DATE_EST"], infer_datetime_format=True)
gamesDf.sort_values(by=['GAME_DATE_EST'], inplace=True, ascending=False)
gamesDf = gamesDf.reset_index()

/var/folders/19/48sc_t714fz_96wrj2kckpy80000gn/T/ipykernel_53586/1343322450.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gamesDf["GAME_DATE_EST"] = pd.to_datetime(gamesDf["GAME_DATE_EST"], infer_datetime_format=True)


In [22]:
rankingsDf["SEASON_ID"] = rankingsDf["SEASON_ID"].apply(lambda x: x - 20000 if x > 20000 else x )
rankingsDf["SEASON_ID"] = rankingsDf["SEASON_ID"].apply(lambda x: x - 10000 if x < 20000 and x > 3000 else x)
rankingsDf["STANDINGSDATE"] = pd.to_datetime(rankingsDf["STANDINGSDATE"], infer_datetime_format=True)

/var/folders/19/48sc_t714fz_96wrj2kckpy80000gn/T/ipykernel_53586/183788237.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rankingsDf["STANDINGSDATE"] = pd.to_datetime(rankingsDf["STANDINGSDATE"], infer_datetime_format=True)


In [23]:
rankingsDfFeatures = [
    "TEAM_ID",
    "SEASON_ID",
    "STANDINGSDATE",
    "G",
    "W",
    "L",
    "W_PCT",
    "HOME_RECORD",
    "ROAD_RECORD",
]
rankingsDf = rankingsDf[rankingsDfFeatures]
rankingsDf

,TEAM_ID,SEASON_ID,STANDINGSDATE,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
0,1610612743,2022,2022-12-22,30,19,11,0.633,10-3,9-8
1,1610612763,2022,2022-12-22,30,19,11,0.633,13-2,6-9
2,1610612740,2022,2022-12-22,31,19,12,0.613,13-4,6-8
3,1610612756,2022,2022-12-22,32,19,13,0.594,14-4,5-9
4,1610612746,2022,2022-12-22,33,19,14,0.576,11-7,8-7
...,...,...,...,...,...,...,...,...,...
210337,1610612765,2013,2014-09-01,82,29,53,0.354,17-24,12-29
210338,1610612738,2013,2014-09-01,82,25,57,0.305,16-25,9-32
210339,1610612753,2013,2014-09-01,82,23,59,0.280,19-22,4-37
210340,1610612755,2013,2014-09-01,82,19,63,0.232,10-31,9-32


## Adding Mean Points Scored From Past 20 Games

In [24]:
meanPpgLast5Home = []
AvgPaLast5Home = []
AvgPpgLast5Away = []
AvgPaLast5Away = []

In [25]:
for i in range(len(gamesDf)):
    date = gamesDf["GAME_DATE_EST"][i]
    homeTeam = gamesDf["HOME_TEAM_ID"][i]
    
    statsHome = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.HOME_TEAM_ID == homeTeam)]

    statsAway = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.VISITOR_TEAM_ID == homeTeam)]

    if not statsHome.empty:
        nPrevGames = len(statsHome.index)
    if statsHome.empty or nPrevGames < 10:
        prevSeason = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.HOME_TEAM_ID == homeTeam)]
        lastSeason10 = prevSeason.iloc[:10]
        avgPpgLast10GamesVal = lastSeason10.PTS_home.mean(axis=0)
        avgPaLast10GamesVal = lastSeason10.PTS_away.mean(axis=0)
    else:
        mostRecent10Games = statsHome.iloc[:10]
        avgPpgLast10GamesVal = mostRecent10Games.PTS_home.mean(axis=0)
        avgPaLast10GamesVal = mostRecent10Games.PTS_away.mean(axis=0)

    meanPpgLast5Home.append(avgPpgLast10GamesVal)
    AvgPaLast5Home.append(avgPaLast10GamesVal)

    if not statsAway.empty:
        nPrevGamesAway = len(statsAway.index)
    if statsAway.empty or nPrevGamesAway < 10:
        prevSeasonAway = gamesDf.loc[(gamesDf.GAME_DATE_EST < date)& (gamesDf.VISITOR_TEAM_ID == homeTeam)]
        lastSeason10Away = prevSeasonAway.iloc[:10]
        avgPpgLast10GamesValAway = lastSeason10Away.PTS_away.mean(axis=0)
        avgPaLast10GamesValAway = lastSeason10Away.PTS_home.mean(axis=0)
    else:
        mostRecent10Games = statsAway.iloc[:10]
        avgPpgLast10GamesValAway = mostRecent10Games.PTS_away.mean(axis=0)
        avgPaLast10GamesValAway = mostRecent10Games.PTS_home.mean(axis=0)

    AvgPpgLast5Away.append(avgPpgLast10GamesValAway)
    AvgPaLast5Away.append(avgPaLast10GamesValAway)

gamesDf["avgppglast5_at_home"] = meanPpgLast5Home
gamesDf["avgpalast5_at_home"] = AvgPaLast5Home
gamesDf["avgppglast5_at_away"] = AvgPpgLast5Away
gamesDf["avgpalast5_at_away"] = AvgPaLast5Away

gamesDf["point_average_last10"] = (gamesDf["avgppglast5_at_home"] + gamesDf["avgppglast5_at_away"]) / 2
gamesDf["point_againts_average_last10"] = (gamesDf["avgpalast5_at_home"] + gamesDf["avgpalast5_at_away"]) / 2

In [26]:
meanPpgLast5HomeAway = []
AvgPaLast5HomeAway = []
AvgPpgLast5AwayAway = []  
AvgPaLast5AwayAway = []

In [27]:
for i in range(len(gamesDf)):
    date = gamesDf["GAME_DATE_EST"][i]
    awayTeam = gamesDf["VISITOR_TEAM_ID"][i]
    
    statsHome = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.HOME_TEAM_ID == awayTeam)]

    statsAway = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.VISITOR_TEAM_ID == awayTeam)]

    if not statsHome.empty:
        nPrevGames = len(statsHome.index)
    if statsHome.empty or nPrevGames < 10:
        prevSeason = gamesDf.loc[(gamesDf.GAME_DATE_EST < date) & (gamesDf.HOME_TEAM_ID == awayTeam)]
        lastSeason10 = prevSeason.iloc[:10]
        avgPpgLast10GamesVal = lastSeason10.PTS_home.mean(axis=0)
        avgPaLast10GamesVal = lastSeason10.PTS_away.mean(axis=0)
    else:
        mostRecent10Games = statsHome.iloc[:10]
        avgPpgLast10GamesVal = mostRecent10Games.PTS_home.mean(axis=0)
        avgPaLast10GamesVal = mostRecent10Games.PTS_away.mean(axis=0)

    meanPpgLast5HomeAway.append(avgPpgLast10GamesVal)
    AvgPaLast5HomeAway.append(avgPaLast10GamesVal)

    if not statsAway.empty:
        nPrevGamesAway = len(statsAway.index)
    if statsAway.empty or nPrevGamesAway < 10:
        prevSeasonAway = gamesDf.loc[(gamesDf.GAME_DATE_EST < date)& (gamesDf.VISITOR_TEAM_ID == awayTeam)]
        lastSeason10Away = prevSeasonAway.iloc[:10]
        avgPpgLast10GamesValAway = lastSeason10Away.PTS_away.mean(axis=0)
        avgPaLast10GamesValAway = lastSeason10Away.PTS_home.mean(axis=0)
    else:
        mostRecent10Games = statsAway.iloc[:10]
        avgPpgLast10GamesValAway = mostRecent10Games.PTS_away.mean(axis=0)
        avgPaLast10GamesValAway = mostRecent10Games.PTS_home.mean(axis=0)

    AvgPpgLast5AwayAway.append(avgPpgLast10GamesValAway)
    AvgPaLast5AwayAway.append(avgPaLast10GamesValAway)

gamesDf["aavgppglast5_at_home"] = meanPpgLast5HomeAway
gamesDf["aavgpalast5_at_home"] = AvgPaLast5HomeAway
gamesDf["aavgppglast5_at_away"] = AvgPpgLast5AwayAway
gamesDf["aavgpalast5_at_away"] = AvgPaLast5AwayAway

gamesDf["away_point_average_last10"] = (gamesDf["aavgppglast5_at_home"] + gamesDf["aavgppglast5_at_away"]) / 2
gamesDf["away_point_againts_average_last10"] = (gamesDf["aavgpalast5_at_home"] + gamesDf["aavgpalast5_at_away"]) / 2

In [28]:
newGamesDf = gamesDf.drop(
    [
        "aavgppglast5_at_home",
        "aavgpalast5_at_home",
        "aavgppglast5_at_away",
        "aavgpalast5_at_away",
        "avgppglast5_at_home",
        "avgpalast5_at_home",
        "avgppglast5_at_away",
        "avgpalast5_at_away",
    ],
    axis=1,
)

In [29]:
newGamesDf

,index,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,...,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PTS_total,point_average_last10,point_againts_average_last10,away_point_average_last10,away_point_againts_average_last10
0,0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,...,0.815,0.321,23.0,44.0,1,243.0,118.40,114.40,108.45,119.75
1,1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,...,0.765,0.333,20.0,37.0,1,232.0,116.05,118.10,113.05,114.95
2,2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,...,0.682,0.433,20.0,46.0,1,220.0,106.70,103.45,113.95,111.80
3,8,2022-12-21,22200472,Final,1610612745,1610612753,2022,1610612745,110.0,0.466,...,0.697,0.297,19.0,45.0,0,226.0,110.85,114.90,108.30,113.95
4,12,2022-12-21,22200476,Final,1610612746,1610612766,2022,1610612746,126.0,0.506,...,0.759,0.290,25.0,40.0,1,231.0,109.30,107.75,111.70,119.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,19280,2003-10-07,10300010,Final,1610612764,1610612752,2003,1610612764,104.0,0.506,...,0.852,0.188,19.0,37.0,1,190.0,NaN,NaN,NaN,NaN
26647,19279,2003-10-07,10300006,Final,1610612747,1610612744,2003,1610612747,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
26648,19285,2003-10-07,10300005,Final,1610612757,1610612745,2003,1610612757,104.0,0.527,...,0.667,0.333,10.0,37.0,1,184.0,NaN,NaN,NaN,NaN
26649,19287,2003-10-06,10300002,Final,1610612763,1610612749,2003,1610612763,105.0,0.494,...,0.700,0.154,20.0,43.0,1,199.0,NaN,NaN,NaN,NaN


In [30]:
newGamesDf.to_csv(r"../data/nbaCleanedData.csv", index=False)